In [1]:
!pip install -r requirements.txt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets, transforms
import os
import wandb
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import random


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [ ]:
data_dir = os.path.join('nature_12K', 'inaturalist_12K')
train_dir = os.path.join(data_dir, 'train')
best_model_path = "best_models"
os.makedirs(best_model_path, exist_ok=True)

print("Data directory:", data_dir)